<a href="https://colab.research.google.com/github/mangalm96/explainablenlp-info259/blob/main/interpret_shap_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import json
import pandas as pd

In [3]:
def generate_shap_df(filepath):
  with open(filepath) as f:
    lines = f.readlines()
  shap_df  = pd.DataFrame()
  for line in lines:
    dictionary = dict()
    if line != '':
      line_data = json.loads(line)
      line_data_df = pd.DataFrame.from_dict(line_data["tokens"])
      if len(shap_df) == 0:
        shap_df = line_data_df
      else:
        shap_df.append(line_data_df)
    shap_df = shap_df.sort_values("token")
  return shap_df

In [4]:
CI_ESRX_target = "/content/gdrive/MyDrive/NLP Stance Detection/xai_atanasova/WTWT/CI_ESRX_target/model_output_shap"
shap_CI_ESRX = generate_shap_df(CI_ESRX_target)

In [5]:
shap_CI_ESRX.head(30)

,token,0,1,2,3
11,[.],-0.060951,0.058714,0.192585,-0.020560
7,[.],-0.019404,-0.048066,0.239455,-0.043079
21,[</s>],-0.049227,0.084209,0.183739,-0.013249
13,[</s>],-0.068591,-0.090330,0.166975,-0.046177
12,[</s>],-0.075020,-0.079719,0.015410,-0.046493
0,[<s>],-0.014689,-0.032746,-0.013662,0.019282
3,[<unk>],-0.031289,-0.202716,0.187441,-0.053296
1,[<unk>],-0.069674,-0.022485,0.023150,0.019070
14,[Cig@@],-0.014407,-0.030730,0.310898,-0.039025
16,[Ex@@],-0.069368,-0.068314,0.088047,-0.011514


In [ ]:
CI_ESRX_notarget = "/content/gdrive/MyDrive/NLP Stance Detection/xai_atanasova/WTWT/CI_ESRX_notarget/model_output_shap"
shap_CI_ESRX_notarget = generate_shap_df(CI_ESRX_notarget)

In [ ]:
shap_CI_ESRX_notarget.head(30)

,token,0,1,2,3
11,[.],-0.060951,0.058714,0.192585,-0.020560
7,[.],-0.019404,-0.048066,0.239455,-0.043079
21,[</s>],-0.049227,0.084209,0.183739,-0.013249
13,[</s>],-0.068591,-0.090330,0.166975,-0.046177
12,[</s>],-0.075020,-0.079719,0.015410,-0.046493
0,[<s>],-0.014689,-0.032746,-0.013662,0.019282
3,[<unk>],-0.031289,-0.202716,0.187441,-0.053296
1,[<unk>],-0.069674,-0.022485,0.023150,0.019070
14,[Cig@@],-0.014407,-0.030730,0.310898,-0.039025
16,[Ex@@],-0.069368,-0.068314,0.088047,-0.011514
